In [2]:
library(tbm)
library(tram)

In [3]:
data_import = function(data_name){
  filename  = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/',data_name,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [4]:
data_massage = function(inputs,labels){
    rownamesInput = rownames(inputs)
    inputs        = do.call(data.frame,lapply(inputs, function(x) replace(x, is.infinite(x),NA)))
    naColumns     = colnames(inputs)[colSums(is.na(inputs))>0]
    noVarCol      = getNonVarCols(inputs)
    removeCols    = c(naColumns,noVarCol)
    inputs        = inputs[ , !(colnames(inputs) %in% removeCols)]
    rownames(inputs) = rownamesInput
    labels$min.log.lambda = unlist(lapply(labels$min.log.lambda,exp))
    labels$max.log.lambda = unlist(lapply(labels$max.log.lambda,exp))
    res        = list()
    res$inputs = inputs
    res$labels = labels
    return(res)
}

In [5]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [6]:
getNonVarCols<-function(data){
    var_columns    = apply(data,2,var)
    resCol         = names(var_columns[var_columns==0.0])
    return(resCol)
}

In [9]:
getaccuracy=function(pred,y_lower,y_higher){
    res = (pred>=y_lower & pred<=y_higher)
    return(res)
}

In [10]:
get_hyper_parameter=function(train,test,y.lower.trn,y.upper.trn,y.lower.tst,y.upper.tst,mboost_grid){
    accuracy = numeric(dim(mboost_grid)[1])
    for(i in 1:2){
        mstop               = mboost_grid[i,1]
        nu                  = mboost_grid[i,2]
        my.surv             = survival::Surv(y.lower.trn,y.upper.trn,type='interval2')
        formula             = as.formula(paste("my.surv ~", paste(colnames(train),collapse="+")))
        trn.data            = data.frame(train,y.lower.trn,y.upper.trn)
        tst.data            = data.frame(test)
        ### With STM - Mboost/Mboost (Additive Smooth Models)
        trn.data$y.lower = trn.data$y.upper = NULL
        trn.data$my.surv = my.surv
        m_mlt            = Survreg(my.surv~1, data = trn.data, dist = "lognormal")
        bm               = stmboost(m_mlt, formula = formula, data = trn.data,control = boost_control(mstop=mstop,nu=nu,trace=TRUE),method = quote(mboost::mboost))
        ### look at in-sample performance
        ##logLik(m_mlt)
        ##plot(risk(bm)) ### this is the negative log-lik
        pred.y.val   = as.numeric(predict(bm,newdata = tst.data,type = "density",q = 0.5))
        accuracy[i]  = sum(mapply(getaccuracy,pred.y.val,y.lower.tst,y.upper.tst))/length(pred.y.val)
        print(pred.y.val)
        print(accuracy[i])
    }
    #mboost_grid$accuracy = accuracy
    #return(mboost_grid)
    return
}

In [11]:
create_data = function(data_name){
    res                 = data_import(data_name)
    inputs              = res$inputs
    labels              = res$labels
    folds               = res$folds
    res_data_massage    = data_massage(inputs,labels)
    inputs              = res_data_massage$inputs
    labels              = res_data_massage$labels
    fold_iter           = unique(folds$fold)
    accuracy_fold       = numeric(length(fold_iter))
    result              = list()
    result$fold_iter    = fold_iter
    result$folds        = folds
    result$inputs       = inputs
    result$labels       = labels
    result$accuracy_fold = accuracy_fold
    return(result)
}

In [12]:
train_test_model  = function(data_name,mboost_grid){
    result        = create_data(data_name)
    folds         = result$folds
    fold_iter     = result$fold_iter
    inputs        = result$inputs
    labels        = result$labels
    accuracy_fold = result$accuracy_fold
    for(i in fold_iter){
        res                 = getXY(i,folds,inputs,labels)
        X                   = res$X
        X.val               = res$X.val
        y.lower             = res$y.lower
        y.lower.val         = res$y.lower.val
        y.upper             = res$y.upper
        y.upper.val         = res$y.upper.val
        train.folds         = cut(seq(1,nrow(X)),breaks=5,labels=FALSE)
        res                 = list()
        for(j in 1:5){
            testIndexes = which(train.folds==j,arr.ind=TRUE)
            X.tst        = X[testIndexes, ]
            X.trn        = X[-testIndexes, ]
            y.lower.trn  = y.lower[-testIndexes,] 
            y.upper.trn  = y.upper[-testIndexes,]
            y.lower.tst  = y.lower[testIndexes,]
            y.upper.tst  = y.upper[testIndexes,]
            res[[j]]     = get_hyper_parameter(X.trn,X.tst,y.lower.trn,y.upper.trn,y.lower.tst,y.upper.tst,mboost_grid)
        }
    }
}

## Hyper-Parameter - mstop and nu

In [13]:
mboost_grid = expand.grid(mstop = seq(1000, 10000, 2000), nu = c(0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01))

In [14]:
data_name_range = c('ATAC_JV_adipose','CTCF_TDH_ENCODE','H3K27ac-H3K4me3_TDHAM_BP','H3K27ac_TDH_some','H3K36me3_AM_immune')

In [ ]:
train_test_model(data_name_range[1],mboost_grid)

[    1] ...................................... -- risk: 361.613 
[   41] ...................................... -- risk: 360.0092 
[   81] ...................................... -- risk: 358.4799 
[  121] ...................................... -- risk: 357.0212 
[  161] ...................................... -- risk: 355.6295 
[  201] ...................................... -- risk: 354.3016 
[  241] ...................................... -- risk: 353.034 
[  281] ...................................... -- risk: 351.823 
[  321] ...................................... -- risk: 350.6666 
[  361] ...................................... -- risk: 349.5617 
[  401] ...................................... -- risk: 348.5056 
[  441] ...................................... -- risk: 347.4961 
[  481] ...................................... -- risk: 346.5307 
[  521] ...................................... -- risk: 345.6076 
[  561] ...................................... -- risk: 344.724 
[  601] ......

Warning message in c.basis(bresponse = response, bshifting = shifting):
“more than one basis contains an intercept term”


 [1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[26] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[51] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[1] NA
[    1] ...................................... -- risk: 361.613 
[   41] ...................................... -- risk: 360.0092 
[   81] ...................................... -- risk: 358.4799 
[  121] ...................................... -- risk: 357.0212 
[  161] ...................................... -- risk: 355.6295 
[  201] ...................................... -- risk: 354.3015 
[  241] ...................................... -- risk: 353.0335 
[  281] ...................................... -- risk: 351.8227 
[  321] ...................................... -- risk: 350.6665 
[  361] ...................................... -- risk: 349.5615 
[  401] ...................................... -- risk: 348.5055 
[  441] ......................................

Warning message in ngradient(y, fit, weights):
“risk increase; decrease stepsize nu”


.......................... -- risk: 9664.903 
[  641] ...................................... -- risk: 9655.793 
[  681] ...................................... -- risk: 9646.958 
[  721] ...................................... -- risk: 9640.811 
[  761] ...................................... -- risk: 9630.794 
[  801] ........

...

Warning message in c.basis(bresponse = response, bshifting = shifting):
“more than one basis contains an intercept term”


 [1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[26] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[51] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[1] NA
[    1] ...................................... -- risk: 360.8191 
[   41] ...................................... -- risk: 359.8192 
[   81] ...................................... -- risk: 358.852 
[  121] ...................................... -- risk: 357.9152 
[  161] ...................................... -- risk: 357.0083 
[  201] ...................................... -- risk: 356.1296 
[  241] ...................................... -- risk: 355.2779 
[  281] ...................................... -- risk: 354.4525 
[  321] ...................................... -- risk: 353.6524 
[  361] ...................................... -- risk: 352.8764 
[  401] ...................................... -- risk: 352.1238 
[  441] ...................................... -- ri

Warning message in c.basis(bresponse = response, bshifting = shifting):
“more than one basis contains an intercept term”


 [1] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[26] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[51] NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
[1] NA
[    1] ...................................... -- risk: 360.5963 
[   41] ...................................... -- risk: 359.535 
[   81] ...................................... -- risk: 358.5041 
[  121] ...................................... -- risk: 357.5028 
[  161] ...................................... -- risk: 356.5299 
[  201] .

Warning message in ngradient(y, fit, weights):
“risk increase; decrease stepsize nu”


In [30]:
result = do.call(cbind,res)
result = cbind(result,apply(result,1,mean))
pr_sel = which.max(result[,6])
X.pr   = prcomp(X, center = TRUE, scale = TRUE)
X.sub  = X.pr$x[,c(1:pr_sel)]
data            = data.frame(X.sub,y.lower,y.upper)
colnames(data)  = c(colnames(X.pr$x)[1:pr_sel],"y_lower","y_upper")
my.surv         = Surv(y.lower,y.upper,type='interval2')
formula         = as.formula(paste("my.surv ~", paste(colnames(X.pr$x)[1:pr_sel],collapse="+")))
sr.fit          = survreg(formula,data=data,dist='lognormal', control = list(maxiter=500))
pred.val.pr     = predict(X.pr,newdata=X.val)
pred.val.pr.sub = data.frame(pred.val.pr[,c(1:pr_sel)])
colnames(pred.val.pr.sub) = colnames(X.pr$x)[1:pr_sel]
pred.y.val       = predict(sr.fit,pred.val.pr.sub)
accuracy_fold[i] = sum(mapply(getaccuracy,pred.y.val,y.lower.val,y.upper.val))/length(pred.y.val)
pred_data           = data.frame(pred.y.val,y.lower.val,y.upper.val)
colnames(pred_data) = c("predict","y.lower","y.upper")
file_name            = paste("../../../../result/",data_name,"/mboost/",i,".csv",sep="")
write.table(pred_data,file_name,sep=",",col.names=NA)
}
jsonAccuracy = toJSON(accuracy_fold)
file_name    = paste("../../../../result/",data_name,"/mboost/accuracy.JSON",sep="")
write(jsonAccuracy, file=file_name)